In [ ]:
#import all the necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


# # #Read the Data 

In [ ]:
data = '../input/brain-mri-images-for-brain-tumor-detection/'
No_brain_tumor = '../input/brain-mri-images-for-brain-tumor-detection/no/'
Yes_brain_tumor = '../input/brain-mri-images-for-brain-tumor-detection/yes/'

In [ ]:
import os

dir_list = [No_brain_tumor, Yes_brain_tumor]
classes = ['No', 'Yes']
filepaths = []
labels = []

for i,j in zip(dir_list, classes):
    filelist = os.listdir(i)
    for name in filelist:
        filepath = os.path.join(i, name)
        filepaths.append(filepath)
        labels.append(j)
        
print('filepaths:', len(filepaths), 'labels:', len(labels))

In [ ]:
file_content=pd.Series(filepaths, name='filepaths')
label_content=pd.Series(labels, name='labels')
df=pd.concat([file_content,label_content], axis=1)
df=pd.DataFrame(np.array(df).reshape(253,2), columns = ['filepaths', 'labels'])
df.head()

In [ ]:
print(df['labels'].value_counts())

# # # Visualize the data

In [ ]:
#visualize brain tumor images
import cv2

plt.figure(figsize=(10,8))
for i in range(15):
    random = np.random.randint(1,len(df))
    plt.subplot(3,5,i+1)
    plt.imshow(cv2.imread(df.loc[random,"filepaths"]))
    plt.title(df.loc[random, "labels"], size = 15, color = "white") 
    plt.xticks([])
    plt.yticks([])
    
plt.show()


# # # Split the data

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df, test_size=0.2, random_state=42)
x_train, x_val = train_test_split(x_train, test_size=0.2, random_state=42)

print(f"train set shape: {x_train.shape}")
print(f"test set shape: {x_test.shape}")
print(f"validation set shape: {x_val.shape}")

# # # Data Augumentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode ='nearest')

validation_datagen = ImageDataGenerator(rescale = 1.0/255.0)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe = x_train,
                                                   x_col = 'filepaths',
                                                   y_col = 'labels',
                                                   target_size=(150,150),
                                                   class_mode='binary',
                                                   batch_size=16)

validation_generator = validation_datagen.flow_from_dataframe(dataframe = x_val,
                                                             x_col = 'filepaths',
                                                             y_col = 'labels',
                                                             target_size = (150,150),
                                                             class_mode='binary',
                                                             batch_size=8)

test_generator = test_datagen.flow_from_dataframe(dataframe = x_test,
                                                  x_col = 'filepaths',
                                                  y_col = 'labels',
                                                  target_size = (150,150),
                                                  class_mode='binary',
                                                  batch_size=8)

# # # Modelling

Lets do the transfer learning from the known model.


Lets use the InceptionResNetv2 for the baseconvolution.


In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', 
                                                     input_shape = (150,150,3),
                                                     include_top=False)


model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate = 0.0001), metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator,
                   epochs=30,
                   steps_per_epoch=10,
                   validation_data = validation_generator,
                   validation_steps=6,
                   verbose=1)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy  = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(15,10))

plt.subplot(2, 2, 1)
plt.plot(accuracy, label = "Training accuracy")
plt.plot(val_accuracy, label="Validation accuracy")
plt.legend()
plt.title("Training vs validation accuracy")


plt.subplot(2,2,2)
plt.plot(loss, label = "Training loss")
plt.plot(val_loss, label="Validation loss")
plt.legend()
plt.title("Training vs validation loss")

plt.show()

In [ ]:
test_accuracy = model.evaluate(test_generator)[1]

In [ ]:
print(test_accuracy
     )

**Accuracy of 92% has been achieved.**

In [ ]:
pred = model.predict(test_generator)
pred = np.argmax(pred, axis=1)
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
y_pred = [labels[k] for k in pred]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print("\n The Classification report is \n")
print(classification_report(x_test.labels, y_pred))
print("\n The Confusion Matrix is \n")
print(confusion_matrix(x_test.labels, y_pred))

In [ ]:
nan